# DecentNet from conv layer

    # additionally needed
    """
    
    position
    activated channels
    connection between channels
    
    """

    
        
        # this layer id
        layer_id = 0
        
        # within this layer, a whole filter can be deactivated
        # within a filter, single channels can be deactivated
        # within this layer, filters can be swapped
     
* pruning actually doesn"t work: https://discuss.pytorch.org/t/pruning-doesnt-affect-speed-nor-memory-for-resnet-101/75814   
* fine tune a pruned model: https://stackoverflow.com/questions/73103144/how-to-fine-tune-the-pruned-model-in-pytorch
* an actual pruning mechanism: https://arxiv.org/pdf/2002.08258.pdf

pip install:
    pytorch_lightning

# imports

In [1]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import math
import warnings

import torch
import numpy as np
import torch.nn.functional as F

import torch
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
# from torch.nn import init
from torch.nn.modules.module import Module
from torch.nn.modules.utils import _single, _pair, _triple, _reverse_repeat_tuple
from torch._torch_docs import reproducibility_notes

from torch.nn.common_types import _size_1_t, _size_2_t, _size_3_t
from typing import Optional, List, Tuple, Union


import sys
sys.path.insert(0, "helper")
sys.path.insert(0, "/helper")
sys.path.insert(0, "./helper")
sys.path.insert(0, "../helper")
print(sys.path)

# own module
from visualisation.feature_map import *

import random

['../helper', './helper', '/helper', 'helper', 'C:\\Users\\Christina\\Documents\\datasceyence\\examples', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\python39.zip', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\DLLs', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy', '', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\pixelssl-0.1.4-py3.9.egg', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\win32', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Christina\\anaconda3\\envs\\chrisy\\lib\\site-packages\\Pythonwin']


# Conv experiments

## conv2d layer (slightly adapted original)

In [ ]:
class _ConvNd(torch.nn.Module):

    __constants__ = ['stride', 'padding', 'dilation', 'groups',
                     'padding_mode', 'output_padding', 'in_channels',
                     'out_channels', 'kernel_size']
    __annotations__ = {'bias': Optional[torch.Tensor]}

    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]) -> Tensor:
        ...

    in_channels: int
    _reversed_padding_repeated_twice: List[int]
    out_channels: int
    kernel_size: Tuple[int, ...]
    stride: Tuple[int, ...]
    padding: Union[str, Tuple[int, ...]]
    dilation: Tuple[int, ...]
    transposed: bool
    output_padding: Tuple[int, ...]
    groups: int
    padding_mode: str
    weight: Tensor
    bias: Optional[Tensor]

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Tuple[int, ...],
                 stride: Tuple[int, ...],
                 padding: Tuple[int, ...],
                 dilation: Tuple[int, ...],
                 transposed: bool,
                 output_padding: Tuple[int, ...],
                 groups: int,
                 bias: bool,
                 padding_mode: str,
                 device=None,
                 dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        print(factory_kwargs)
        super().__init__()
        if groups <= 0:
            raise ValueError('groups must be a positive integer')
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        valid_padding_strings = {'same', 'valid'}
        if isinstance(padding, str):
            if padding not in valid_padding_strings:
                raise ValueError(
                    "Invalid padding string {!r}, should be one of {}".format(
                        padding, valid_padding_strings))
            if padding == 'same' and any(s != 1 for s in stride):
                raise ValueError("padding='same' is not supported for strided convolutions")

        valid_padding_modes = {'zeros', 'reflect', 'replicate', 'circular'}
        if padding_mode not in valid_padding_modes:
            raise ValueError("padding_mode must be one of {}, but got padding_mode='{}'".format(
                valid_padding_modes, padding_mode))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        self.padding_mode = padding_mode
        # `_reversed_padding_repeated_twice` is the padding to be passed to
        # `F.pad` if needed (e.g., for non-zero padding types that are
        # implemented as two ops: padding + conv). `F.pad` accepts paddings in
        # reverse order than the dimension.
        if isinstance(self.padding, str):
            self._reversed_padding_repeated_twice = [0, 0] * len(kernel_size)
            if padding == 'same':
                for d, k, i in zip(dilation, kernel_size,
                                   range(len(kernel_size) - 1, -1, -1)):
                    total_padding = d * (k - 1)
                    left_pad = total_padding // 2
                    self._reversed_padding_repeated_twice[2 * i] = left_pad
                    self._reversed_padding_repeated_twice[2 * i + 1] = (
                        total_padding - left_pad)
        else:
            self._reversed_padding_repeated_twice = _reverse_repeat_tuple(self.padding, 2)

        if transposed:
            self.weight = Parameter(torch.empty(
                (in_channels, out_channels // groups, *kernel_size), **factory_kwargs))
            #self.importance = Parameter(torch.empty(
            #    (in_channels, out_channels // groups), **factory_kwargs))
            
        else:
            self.weight = Parameter(torch.empty(
                (out_channels, in_channels // groups, *kernel_size), **factory_kwargs))
            #self.importance = Parameter(torch.empty(
            #    (out_channels, in_channels // groups), **factory_kwargs))
            
        if bias:
            self.bias = Parameter(torch.empty(out_channels, **factory_kwargs))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()


class CustomConv2d(_ConvNd):
    
    # additionally needed
    """
    
    position
    activated channels
    connection between channels
    
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: _size_2_t,
        stride: _size_2_t = 1,
        padding: Union[str, _size_2_t] = 0,
        dilation: _size_2_t = 1,
        groups: int = 1,
        bias: bool = True,
        padding_mode: str = 'zeros',  # TODO: refine this type
        device=None,
        dtype=None
    ) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        kernel_size_ = _pair(kernel_size)
        stride_ = stride #_pair(stride)
        padding_ = padding if isinstance(padding, str) else _pair(padding)
        dilation_ = _pair(dilation)
        super().__init__(
            in_channels, out_channels, kernel_size_, stride_, padding_, dilation_,
            False, _pair(0), groups, bias, padding_mode, **factory_kwargs)
        
        # this layer id
        layer_id = 0
        
        # within this layer, a whole filter can be deactivated
        # within a filter, single channels can be deactivated
        # within this layer, filters can be swapped

    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(k), 1/sqrt(k)), where k = weight.size(1) * prod(*kernel_size)
        # For more details see: https://github.com/pytorch/pytorch/issues/15314#issuecomment-477448573
        torch.nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            if fan_in != 0:
                bound = 1 / math.sqrt(fan_in)
                init.uniform_(self.bias, -bound, bound)

    def extra_repr(self):
        s = ('{in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        if self.padding_mode != 'zeros':
            s += ', padding_mode={padding_mode}'
        return s.format(**self.__dict__)

    def __setstate__(self, state):
        super().__setstate__(state)
        if not hasattr(self, 'padding_mode'):
            self.padding_mode = 'zeros'

    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]):
        if self.padding_mode != 'zeros':
            return F.conv2d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            weight, bias, self.stride,
                            _pair(0), self.dilation, self.groups)
        
        # this is written in c++ - try not to change ...
        print(self.stride)
        return F.conv2d(input, weight, bias, self.stride,
                        self.padding, self.dilation, self.groups)

    def forward(self, input: Tensor) -> Tensor:
        return self._conv_forward(input, self.weight, self.bias)



## model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = CustomConv2d(1, 32, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv2 = CustomConv2d(32, 64, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv3 = CustomConv2d(64, 128, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv1x1 = CustomConv2d(128, 10, kernel_size=1, stride=1, padding=0, dilation=1)
        
        self.K = 100 
        self.L = 10 # last one
        self.num_of_bases = 1 # 3rd dim
        
        if False:
            self.conv1 = Conv2d(1, 32, kernel_size=3, stride=1, padding=0, dilation=3)
            self.conv2 = Conv2d(32, 64, kernel_size=3, stride=1, padding=0, dilation=3)
            self.conv3 = Conv2d(64, 128, kernel_size=3, stride=1, padding=0, dilation=3)
            self.conv1x1 = Conv2d(128, 10, kernel_size=1, stride=1, padding=0, dilation=1)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        # 4x16384
        # self.fc1 = nn.Linear(10*10*10, 10)
        #self.fc2 = nn.Linear(10, 10)
        
        #self.flat = nn.Flatten()
        
        self.fc3 = nn.Linear(10, 10)
        
        self.mish1 = torch.nn.Mish()
        self.mish2 = torch.nn.Mish()
        self.mish3 = torch.nn.Mish()
        self.mish1x1 = torch.nn.Mish()
        
        #self.sub_concept_pooling = nn.modules.MaxPool2d((self.K, 1), stride=(1,1))
        #self.instance_pooling = nn.modules.MaxPool2d((opt.num_of_bases, 1), stride=(1,1))
        
        self.flat = nn.Flatten()
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.mish1(x)
        
        x = self.conv2(x)
        x = self.mish2(x)
        
        x = self.conv3(x)
        x = self.mish3(x)
        
        x = self.conv1x1(x)
        x = self.mish1x1(x)
        
        # print(x.shape)
        
        #x = F.max_pool2d(x, 2)
        #x = self.dropout1(x)
        
        #print(x.size())
        #print(x.size()[2:])
        
        x = F.avg_pool2d(x, kernel_size=x.size()[2:])
        
        x = x.reshape(x.size(0), -1)
        x = self.fc3(x)
        
        # x = self.flat(x)
        
        #x = self.fc2(x)
        
        
        #x = x.view(-1, self.L, self.K, 10)
        
        # input, kernel_size, stride, padding, dilation, ceil_mode
        #x = self.sub_concept_pooling(x).view(-1, self.L, self.num_of_bases).permute(0,2,1).unsqueeze(1)
        
        # output = F.sigmoid(x)
        # x = x.reshape(x.size(0), -1)
        
        #x = torch.flatten(x, 1)
        # x = self.fc1(x)
        #x = F.relu(x)
        
        #x = self.dropout2(x)
        #x = self.fc2(x)
        #output = F.log_softmax(x, dim=1)
        
        x = self.sigmoid(x)
        
        return x





## normal run

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for i_batch, (data, target) in enumerate(train_loader):
        
        
        
        data, target = data.to(device), target.unsqueeze(1) # .to(device)
        optimizer.zero_grad()
        output = model(data)
        
        target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
        
        if i_batch == -1:
            print(data.shape) # torch.Size([4, 1, 28, 28])
            print(target)
            """
            tensor([[8],
            [7],
            [2],
            [7]])
            """
            print(target_multi_hot)
            """
            tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
            [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')
            """
        
        loss = F.binary_cross_entropy(output, target_multi_hot)
        loss.backward()
        optimizer.step()
        if i_batch % (args.log_interval*1000) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i_batch * len(data), len(train_loader.dataset),
                100. * i_batch / len(train_loader), loss.item()))
            


def test(args, model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.unsqueeze(1) # .to(device)
            
            output = model(data)
            
            target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
            test_loss += F.binary_cross_entropy(output, target_multi_hot, reduction='mean').item()
        
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to(device).view_as(pred)).sum().item()
            
            """
            if i == 0 and epoch % args.log_interval == 0:
            # if False: # i == 0:
                print(data.shape)
                layer = model.conv1x1 # model.conv1[0] # model.stage2[0].branch1[2] # model.fusion_layer # conv
                # run feature map
                dd = FeatureMap(model=model, layer=layer, device=device, iterations=None, lr=None)
                dd.run(data)
                dd.plot(path=f"example_results/feature_map_{epoch}.png")
                """

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

class Parser():
    def __init__(self):
        self.batch_size = 128
        self.test_batch_size = 1
        self.epochs = 100
        self.lr = 0.1
        self.gamma = 0.7
        self.log_interval = 5
        self.save_model = True
        

def main_train():
    # Training settings
    args = Parser()
    
    if True:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if device == torch.device("cuda"):
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('example_data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('example_data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr)

    #scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader, epoch)
        #scheduler.step()
        
        
        if args.save_model and epoch % args.log_interval == 0:
            torch.save(model.state_dict(), f"example_results/mnist_cnn_{epoch}.ckpt")


def main_test():
    # Training settings
    args = Parser()
    
    if True:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    test_kwargs = {'batch_size': args.test_batch_size}
    if device == torch.device("cuda"):
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset2 = datasets.MNIST('example_data', train=False,
                       transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)

    if True:
        model.load_state_dict(torch.load("example_results/mnist_cnn_5.ckpt"))
    else:
        model.load_state_dict(torch.load("example_results/pruned_model.ckpt"))
    

    # model = torch.load(model.state_dict(), "example_results/mnist_cnn_30.ckpt")
    if False:
        test(args, model, device, test_loader, 0)
    
    return model
        
    
            


In [ ]:
# main_train()

In [ ]:
# model_to_prune= main_test()

In [ ]:
# print(list(model_to_prune.named_buffers()))

# DecentNet trial and error

In [2]:
labels = torch.tensor([1, 0, 5, 2])
labels = labels.unsqueeze(0)

target = torch.zeros(labels.size(0), 10).scatter_(1, labels, 1.)
print(target)
print(labels)

tensor([[1., 1., 1., 0., 0., 1., 0., 0., 0., 0.]])
tensor([[1, 0, 5, 2]])


In [3]:
# Two lists of data
ms_in = [1, 2, 4]
ns_in = [2, 4, 8]

ms = [2, 2, 2, 5, 9, 4]
ns = [4, 2, 2, 3, 6, 8]

# Find the indices (IDs) of pairs that exist in both lists
common_pairs = [[f, x] for f, (item1, item2) in enumerate(zip(ms_in, ns_in)) for x, (item3, item4) in enumerate(zip(ms, ns)) if (item1==item3 and item2==item4)]

# Print the common pairs
for pair in common_pairs:
    print(f"Common pair at indices {pair}: {ms_in[pair[0]], ms[pair[1]]}, {ns_in[pair[0]], ns[pair[1]]}")
    
a = np.array(common_pairs)
f_ids = a[:,0]
x_ids = a[:,1]

print(f_ids)
print(x_ids)

Common pair at indices [1, 0]: (2, 2), (4, 4)
Common pair at indices [2, 5]: (4, 4), (8, 8)
[1 2]
[0 5]


## DataSet

## X

In [4]:
class X:
    def __init__(self, value, ms, ns):
        
        self.ms = ms # list of integers
        self.ns = ns # list of integers
        self.channels = value # list of tensors
                
    def set(self, value, ms, ns):
        self.ms = ms
        self.ns = ns
        self.channels = value
    
    def get(self):
        return self.channels, self.m, self.n
    
    def __str__(self):
        # amout of channels need to have same length as m and n lists
        return 'X(channels: ' + str(self.channels.shape) +' at positions: ms= ' + ', '.join(str(m.item()) for m in self.ms) + ', ns= ' + ', '.join(str(n.item()) for n in self.ns) + ')'
    
    
    __repr__ = __str__

  

## DecentFilter
* conv2d problem: https://stackoverflow.com/questions/61269421/expected-stride-to-be-a-single-integer-value-or-a-list-of-1-values-to-match-the

In [16]:
class DecentFilter(torch.nn.Module):
    # convolution happens in here
    
    def __init__(self, ms_in, ns_in, m_out, n_out, n_channels=1,
                 kernel_size=3, 
                 stride=1, 
                 padding=0, 
                 padding_mode="zeros",
                 dilation=3, 
                 transposed=None, 
                 device=None, 
                 dtype=None):
        
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        
        
            
        # conv 
        kernel_size = _pair(kernel_size)
        stride = stride
        padding = padding if isinstance(padding, str) else _pair(padding)
        dilation = _pair(dilation)
        
        
        valid_padding_strings = {'same', 'valid'}
        if isinstance(padding, str):
            if padding not in valid_padding_strings:
                raise ValueError(
                    "Invalid padding string {!r}, should be one of {}".format(
                        padding, valid_padding_strings))
            if padding == 'same' and any(s != 1 for s in stride):
                raise ValueError("padding='same' is not supported for strided convolutions")

        valid_padding_modes = {'zeros', 'reflect', 'replicate', 'circular'}
        if padding_mode not in valid_padding_modes:
            raise ValueError("padding_mode must be one of {}, but got padding_mode='{}'".format(
                valid_padding_modes, padding_mode))
            
        self.n_channels = n_channels
        # self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.padding_mode = padding_mode
        
        # position
        self.ms_in = ms_in # list
        self.ns_in = ns_in # list
        self.m_out = m_out # single integer
        self.n_out = n_out # single integer
        
        # weight
        # filters x channels x kernel x kernel
        #self.weights = torch.autograd.Variable(torch.randn(1,n_channels,*kernel_size)).to("cuda")
        # self.weights = torch.nn.Parameter(torch.randn(1,n_channels,*kernel_size))
        self.weights = torch.nn.Parameter(torch.empty((1, n_channels, *kernel_size), **factory_kwargs))
            
        # bias    
        if False: 
            # bias:
            # where should the bias be???
            self.bias = Parameter(torch.empty(1, **factory_kwargs))
        else:
            #self.bias = False
            self.register_parameter('bias', None)
        
        # reset weights and bias in filter
        self.reset_parameters()
            
    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(k), 1/sqrt(k)), where k = weight.size(1) * prod(*kernel_size)
        # For more details see: https://github.com/pytorch/pytorch/issues/15314#issuecomment-477448573
        torch.nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5))        
        
        
    def forward(self, x:X) -> Tensor:
        
        # weights = 1 filter x channels x kernel x kernel
        # x = batch x channels x width x height
        
        # filter channels and weights based on the ms and ns (input)
        tmp_ms = x.ms
        tmp_ns = x.ns
        
        tmp_ms_in = self.ms_in
        tmp_ns_in = self.ns_in
        
        # Two lists of data
        #ms_in = [1, 2, 3, 4, 5]
        #ns_in = [2, 4, 4, 8, 3]

        #ms = [2, 2, 2, 5, 4]
        #ns = [4, 2, 2, 3, 8]

        # Find the indices (IDs) of pairs that exist in both lists
        common_pairs = [[f, x] for f, (item1, item2) in enumerate(zip(tmp_ms_in, tmp_ns_in)) for x, (item3, item4) in enumerate(zip(tmp_ms, tmp_ns)) if (item1==item3 and item2==item4)]
        
        if False:
            print(common_pairs)
            print(len(self.ms_in))
            print(len(self.ns_in))
            print(len(x.ms))
            print(len(x.ns))

            for pair in common_pairs:
                print(f"Common pair at indices {pair}: {tmp_ms_in[pair[0]], tmp_ms[pair[1]]}, {tmp_ns_in[pair[0]], tmp_ns[pair[1]]}")
        
        a = np.array(common_pairs)
        f_ids = a[:,0]
        x_ids = a[:,1]
        
        tmp_x = x.channels[:, x_ids, :, :]
        tmp_f = self.weights[:, f_ids, :, :]
        
        if self.padding_mode != 'zeros':
            x_channels = torch.nn.functional.conv2d(F.pad(tmp_x, self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            tmp_f, self.bias, self.stride,
                            _pair(0), self.dilation, groups=1)
        else:
            # this is written in c++
            x_channels = torch.nn.functional.conv2d(tmp_x, tmp_f, self.bias, self.stride, self.padding, self.dilation, groups=1)
        
        # print(x_channels.shape, "- batch x filters x width x height")        
        return x_channels
            
    
    def update(self):
        # channel deactivation
        # require_grad = False/True for each channel
        
        #deactivate_ids = [1, 2, 6]
        
        #self.active[deactivate_ids] = False
        
        print("updating here")
        
        remove_ids = random.sample(range(0, 8), 5) # this is actually the keep ids rn
        self.weights = torch.nn.Parameter(self.weights[:, remove_ids, :, :])
        # torch.nn.Parameter(torch.empty((1, n_channels, *kernel_size), **factory_kwargs))
        
        self.ms_in = [self.ms_in[i] for i in remove_ids] # self.ms_in[remove_ids]
        self.ns_in = [self.ns_in[i] for i in remove_ids] # self.ns_in[remove_ids]
        
        
        
        
        #print("weight")
        #print(self.weight.shape)
        
        #print(self.weight[:,self.active,:,:].shape)
        
        # this is totally wrong - iterative will break after first iteration
        
        #print()
        
        # Good to hear it’s working, although I would think you’ll get an error at some point in your code, as the cuda() call creates a non-leaf tensor.
        #self.weight = torch.nn.Parameter(  self.weight[:,self.active,:,:] ) # .detach().cpu().numpy()
        #self.weight = self.weight.cuda()
        
        
        #print(self.weight.shape)
        #print(self.active)
    
    def set_position_and_value(self, value, m_out, n_out):
        self.weights = value # weights in this filter
        self.m_out = m_out # single integer
        self.n_out = n_out # single integer
        
    
    def get_position_and_value(self):
        return self.weights, self.m_out, self.n_out
    
    def __str__(self):
        return 'DecentFilter(weights: ' + str(self.weights.shape) + ' at position: m_out=' + str(self.m_out) + ', n_out=' + str(self.n_out) + ')' + \
    '\n with inputs: ms_in= ' + ', '.join(str(m.item()) for m in self.ms_in) + ', ns_in= ' + ', '.join(str(n.item()) for n in self.ns_in) + ')'
    __repr__ = __str__
    
        
        

    
    
    

            


    

## DecentLayer

In [17]:
class DecentLayer(torch.nn.Module):
    __constants__ = ['stride', 'padding', 'dilation', 'groups',
                     'padding_mode', 'n_channels', #  'output_padding',
                     'n_filters', 'kernel_size']
    __annotations__ = {'bias': Optional[torch.Tensor]}
        
        
    def __init__(self, ms_in:list, ns_in:list, n_channels:int, n_filters:int,
                 kernel_size: _size_2_t,   # not in use
                 stride: _size_2_t = 1,  # not in use
                 padding: Union[str, _size_2_t] = 0,  # not in use
                 dilation: _size_2_t = 1,  # not in use
                 transposed: bool = False,  # not in use
                 #output_padding: Tuple[int, ...] = _pair(0),
                 #groups: int = 1,
                 bias: bool = True,  # not in use
                 padding_mode: str = "zeros",  # not in use
                 device=None,  # not in use
                 dtype=None) -> None:
        
        super().__init__()
        
        # position
        self.ms_in = ms_in
        self.ns_in = ns_in

        # use techniques from coo matrix
        self.geometry_array = np.full(81, np.nan)
        # plus 1 here cause of to_sparse array
        self.geometry_array[0:n_filters] = range(1,n_filters+1)
        np.random.shuffle(self.geometry_array)
        self.geometry_array = self.geometry_array.reshape((9,9), order='C')
        self.geometry_array = torch.tensor(self.geometry_array)
        self.geometry_array = self.geometry_array.to_sparse(sparse_dim=2).to("cuda")

        print(self.geometry_array)
        print(self.geometry_array.values())

        self.filter_list = torch.nn.ModuleList([])
        for i_filter in range(n_filters):
            # minus 1 here cause of to_sparse array
            index = (self.geometry_array.values()-1 == i_filter).nonzero(as_tuple=True)[0]
            m_out = self.geometry_array.indices()[0][index]
            n_out = self.geometry_array.indices()[1][index]
            f = DecentFilter(ms_in, ns_in, m_out, n_out, n_channels)
            self.filter_list.append(f)
            # self.register_parameter(f"filter {i_filter}", f.weights)
            
            #torch.nn.Parameter(torch.empty((1, n_channels, *kernel_size), **factory_kwargs))
        
    def sparcify(self) -> None:
        # pruning based on a metric
        
        # delete layer with id
        # delete channels in each layer with id
        
        # change positions
        # change
        
        pass
    
    def reset_parameters(self) -> None:
        pass
        

    def extra_repr(self):
        """
        s = ('{in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        if self.padding_mode != 'zeros':
            s += ', padding_mode={padding_mode}'
        return s.format(**self.__dict__)
        """
        return None

    def __setstate__(self, state):
        super().__setstate__(state)
        if not hasattr(self, 'padding_mode'):
            self.padding_mode = 'zeros'
            
    def update(self):
        for f in self.filter_list:
            f.update()
        
    def forward(self, x: X) -> Tensor:
        
        # calculate output for each filter
        output_list = []
        m_list = []
        n_list = []
        for f in self.filter_list:
            # output = filter(input)
            output_list.append(f(x))
            m_list.append(f.m_out)
            n_list.append(f.n_out)
        x.ms = m_list
        x.ns = n_list
        x.channels = torch.cat(output_list, dim=1)
        return x
    
    def get_filter_positions(self):
        
        ms_out = []
        ns_out = []
        for f in self.filter_list:
            ms_out.append(f.m_out)
            ns_out.append(f.n_out)
        
        return ms_out, ns_out
             

    

            

## DecentNet

In [18]:
class DecentNet(nn.Module):
    def __init__(self, dim=[1, 8, 16, 24], n_classes=10) -> None:
        super(DecentNet, self).__init__()
        
        dim.append(n_classes)
        #dim = [1, 32, 48, 64, 10]        
        assert not any(i > 81 for i in dim), "filters need to be less than 81"
        
        # backbone
        m0 = [torch.tensor(0)]
        n0 = [torch.tensor(0)]
        self.decent1 = DecentLayer(m0, n0, dim[0], dim[1], kernel_size=3, stride=1, padding=0, dilation=3)
        m1,n1 = self.decent1.get_filter_positions()
        self.decent2 = DecentLayer(m1, n1, dim[1], dim[2], kernel_size=3, stride=1, padding=0, dilation=3)
        m2,n2 = self.decent2.get_filter_positions()
        self.decent3 = DecentLayer(m2, n2, dim[2], dim[3], kernel_size=3, stride=1, padding=0, dilation=3)
        m3,n3 = self.decent3.get_filter_positions()
        self.decent1x1 = DecentLayer(m3, n3, dim[3], dim[-1], kernel_size=1, stride=1, padding=0, dilation=1)
        
        # head
        self.fc = torch.nn.Linear(dim[-1], dim[-1])
    
        # activation
        self.mish1 = torch.nn.Mish()
        self.mish2 = torch.nn.Mish()
        self.mish3 = torch.nn.Mish()
        self.mish1x1 = torch.nn.Mish()
        
        # bias
        self.bias1 = torch.nn.InstanceNorm2d(dim[1])
        self.bias2 = torch.nn.InstanceNorm2d(dim[2])
        self.bias3 = torch.nn.InstanceNorm2d(dim[3])
        self.bias1x1 = torch.nn.InstanceNorm2d(dim[-1])
        
        # activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        #print(x)
        
        x = self.decent1(x)
        x.channels = self.mish1(x.channels)
        x.channels = self.bias1(x.channels)
        
        #print(x)
        
        x = self.decent2(x)
        x.channels = self.mish2(x.channels)
        x.channels = self.bias2(x.channels)
        
        #print(x)
        
        x = self.decent3(x)
        x.channels = self.mish3(x.channels)
        x.channels = self.bias3(x.channels)
        
        #print(x)
        
        x = self.decent1x1(x)
        x.channels = self.mish1x1(x.channels)
        x.channels = self.bias1x1(x.channels)
        
        #print(x)
        
        # global max pooling for MIL
        x.channels = F.max_pool2d(x.channels, kernel_size=x.channels.size()[2:])
        
        x.channels = x.channels.reshape(x.channels.size(0), -1)
        x.channels = self.fc(x.channels) 
        
        x.channels = self.sigmoid(x.channels)
        
        return x.channels
    
    def update(self):
        #self.decent1.update()
        self.decent2.update()
        self.decent3.update()
        self.decent1x1.update()
        
        # measurement for updating
        
        # update layer by layer
    
        


In [8]:
#DecentNet(n_classes=5)

## Run

In [33]:
tmp = torch.autograd.Variable(torch.randn(5, 2, 30, 30)) # batch x channel x width x height
# dense_input.shape

# todo: ms need to have same size as channel

X(tmp, [torch.tensor(5), torch.tensor(7)], [torch.tensor(3), torch.tensor(12)])

X(channels: torch.Size([5, 2, 30, 30]) at positions: ms= 5, 7, ns= 3, 12)

### Lightning version

In [25]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import os
import torchmetrics

class DecentLightning(pl.LightningModule):
    def __init__(self, kwargs):
        super().__init__()
        
        model_kwargs = kwargs['model_kwargs']
        
        self.n_classes = model_kwargs["n_classes"]
        self.model = DecentNet(n_classes=self.n_classes).to("cuda")
        self.criterion = model_kwargs["criterion"]
        self.optimizer = model_kwargs["optimizer"]
        self.lr = model_kwargs["lr"]
        self.cc_weight = model_kwargs["cc_weight"]
        
        
        self.save_hyperparameters()
        
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=self.n_classes)
        self.f1score = torchmetrics.F1Score(task="multiclass", num_classes=self.n_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # 1       
        loss = self._loss_n_metrics(batch, mode="train")
        if batch_idx < 5:
            print("training_step", batch_idx)
        #loss = torch.tensor(1)
        return loss
    
    
    def validation_step(self, batch, batch_idx):
        # 2
        self._loss_n_metrics(batch, mode="val")
        if batch_idx < 5:
            print("validation_step", batch_idx)
        #self._loss_n_metrics(batch, mode="val")
        

        
    
    def on_validation_epoch_end(self):
        print("on_validation_epoch_end")
        # 3
        
        """
        for parameter in self.parameters():
            print("parameter")
            print(parameter)
        """ 
        #return
        
        print(self.model)
        
        self.model.update()
        
        print("*"*50)
        print("*"*50)
        print("*"*50)
        print("model is updated now")
        print("*"*50)
        print("*"*50)
        print("*"*50)
        
        print(self.model)
        
        pass
    
    def on_train_epoch_end(self):
        print("on_train_epoch_end")
        # 4
        

        
        pass

    def configure_optimizers(self):
        print("configure_optimizers")
        
        if self.optimizer == "adamw":
            optimizer = optim.AdamW(self.parameters(), lr=self.lr)
            lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)
            return [optimizer], [lr_scheduler]
        else:
            return optim.SGD(self.parameters(), lr=self.lr)
        
    def get_cc(self):
        # compute connection cost
        # adapted from BIMT: https://github.com/KindXiaoming/BIMT/blob/main/mnist_3.5.ipynb
        # def get_cc(self, weight_factor=2.0, bias_penalize=True, no_penalize_last=False):
        # https://stackoverflow.com/questions/74086766/how-to-find-total-cost-of-each-path-in-graph-using-dictionary-in-python
        cc = 0
        """
        num_linear = len(self.linears)
        for i in range(num_linear):
            if i == num_linear - 1 and no_penalize_last:
                weight_factor = 0.
            biolinear = self.linears[i]
            dist = torch.sum(torch.abs(biolinear.out_coordinates.unsqueeze(dim=1) - biolinear.in_coordinates.unsqueeze(dim=0)),dim=2)
            cc += torch.mean(torch.abs(biolinear.linear.weight)*(weight_factor*dist+self.l0))
            if bias_penalize == True:
                cc += torch.mean(torch.abs(biolinear.linear.bias)*(self.l0))
        if self.token_embedding:
            cc += torch.mean(torch.abs(self.embedding)*(self.l0))
            #pass
        """
        return cc
    
    def _loss_n_metrics(self, batch, mode="train"):
        
        img, ground_truth = batch
        # make it an X object
        img = X(img.to("cuda"), [torch.tensor(0)], [torch.tensor(0)])
        
        model_output = self(img) # cause of the forward function
        
        ground_truth = ground_truth.unsqueeze(1)
        
        ground_truth_multi_hot = torch.zeros(ground_truth.size(0), self.n_classes).scatter_(1, ground_truth.to("cpu"), 1.).to("cuda")
        
        # this needs fixing
        # ground_truth_multi_hot = torch.zeros(ground_truth.size(0), 10).to("cuda").scatter_(torch.tensor(1).to("cuda"), ground_truth.to("cuda"), torch.tensor(1.).to("cuda")).to("cuda")
        
        loss = self.criterion(model_output, ground_truth_multi_hot)
        cc = self.get_cc() * self.cc_weight
        # from BIMT
        # loss_train = loss_fn(mlp(x.to(device)), one_hots[label])
        # cc = mlp.get_cc(weight_factor=2.0, no_penalize_last=True)
        # total_loss = loss_train + lamb*cc
        
        
        acc = self.accuracy(preds=model_output, target=ground_truth_multi_hot) # (model_output.argmax(dim=-1) == ground_truth).float().mean()
        f1score = self.f1score(preds=model_output, target=ground_truth_multi_hot) 
        
        self.log(f'{mode}_loss', loss, on_step=False, on_epoch=True)
        self.log(f'{mode}_cc', cc, on_step=False, on_epoch=True)
        self.log(f'{mode}_acc', acc, on_step=False, on_epoch=True)
        self.log(f'{mode}_f1score', f1score, on_step=False, on_epoch=True)
        
        # loss + connection cost term
        return loss + cc



In [26]:
def dev_routine(**kwargs):
    
    print("train kwargs", kwargs['train_kwargs'])
    print("model kwargs", kwargs['model_kwargs'])
    
    train_kwargs = kwargs['train_kwargs']
    
    transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_set = datasets.MNIST('example_data', train=True, download=True, transform=transform)
    val_set = datasets.MNIST('example_data', train=False, transform=transform)
    
    train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=train_kwargs["batch_size"])
    val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=train_kwargs["batch_size"])
    
    logger = CSVLogger("example_results/lightning_logs", name="tmp_exp")
    trainer = pl.Trainer(default_root_dir=os.path.join(train_kwargs["ckpt_path"], "example_results"),
                         accelerator="gpu" if str(train_kwargs["device"]).startswith("cuda") else "cpu",
                         devices=1,
                         logger=logger,
                         check_val_every_n_epoch=1,
                         max_epochs=train_kwargs["epochs"],
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                                    LearningRateMonitor("epoch")])
    
    
    
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(train_kwargs["ckpt_path"], "DecentNet.ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        model = DecentLightning.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
    else:
        pl.seed_everything(19) # To be reproducable
        
        # Initialize the LightningModule and LightningDataModule
        model = DecentLightning(kwargs)
        
        #print(model)

        # Train the model using a Trainer
        trainer.fit(model, train_dataloader, val_dataloader)
        
        #print(model)
        
        model = DecentLightning.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"test": test_result[0]["test_acc"], "val": val_result[0]["test_acc"]}

    return model, result
    

In [27]:
model, results = dev_routine(model_kwargs={
                                'n_classes': 10,
                                'criterion': torch.nn.BCELoss(),
                                'optimizer': "adamw",
                                'lr': 0.001,
                                'cc_weight': 0.2,
                            },
                            train_kwargs={
                                'epochs': 1,
                                'batch_size': 16,
                                # 'test_batch_size': 1,
                                'ckpt_path': "",
                                'device': "cuda"
                            }
                            )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 19
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


train kwargs {'epochs': 1, 'batch_size': 16, 'ckpt_path': '', 'device': 'cuda'}
model kwargs {'n_classes': 10, 'criterion': BCELoss(), 'optimizer': 'adamw', 'lr': 0.001, 'cc_weight': 0.2}
tensor(indices=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
                        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,
                        4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6,
                        6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8,
                        8, 8, 8, 8, 8],
                       [0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0,
                        1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1,
                        2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2,
                        3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3,
                        4, 5, 6, 7, 8]]),
       values=tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 


  | Name      | Type               | Params
-------------------------------------------------
0 | model     | DecentNet          | 7.0 K 
1 | criterion | BCELoss            | 0     
2 | accuracy  | MulticlassAccuracy | 0     
3 | f1score   | MulticlassF1Score  | 0     
-------------------------------------------------
7.0 K     Trainable params
0         Non-trainable params
7.0 K     Total params
0.028     Total estimated model params size (MB)


configure_optimizers
Sanity Checking DataLoader 0: 100%|██████████████████████████████████████████████████████| 2/2 [00:02<00:00,  0.93it/s]on_validation_epoch_end
DecentNet(
  (decent1): DecentLayer(
    (filter_list): ModuleList(
      (0): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([3], device='cuda:0'), n_out=tensor([8], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (1): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([2], device='cuda:0'), n_out=tensor([1], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (2): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([6], device='cuda:0'), n_out=tensor([3], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (3): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([2], device='cuda:0'), n_out=tensor([8], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (4): DecentFilter(weights: tor

PermissionError: [Errno 13] Permission denied: 'C:/Users/Christina/Documents/datasceyence/examples'

TypeError: __init__() missing 1 required positional argument: 'kwargs'

### normal train without lightning

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for i_batch, (data, target) in enumerate(train_loader):
        
        
        data, target = data.to(device), target.unsqueeze(1) # .to(device)
        optimizer.zero_grad()
        output = model(data)
        
        
        target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
        
        if i_batch == 5:
            
            
            #model.update()
            
            #print(data.shape) # torch.Size([4, 1, 28, 28])
            #print(target)
            """
            tensor([[8],
            [7],
            [2],
            [7]])
            """
            #print(target_multi_hot)
            """
            tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
            [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')
            """
            pass
        
        
        loss = F.binary_cross_entropy(output, target_multi_hot)
        loss.backward()
        optimizer.step()
        if i_batch % (args.log_interval*1000) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i_batch * len(data), len(train_loader.dataset),
                100. * i_batch / len(train_loader), loss.item()))
            
            # model.update()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.unsqueeze(1) # .to(device)
            
            output = model(data)
            
            target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
            test_loss += F.binary_cross_entropy(output, target_multi_hot, reduction='mean').item()
        
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to(device).view_as(pred)).sum().item()
            
            if False: # i == 0:
                print(data.shape)
                layer = model.conv1x1 # model.conv1[0] # model.stage2[0].branch1[2] # model.fusion_layer # conv
                # run feature map
                dd = FeatureMap(model=model, layer=layer, device=device, iterations=None, lr=None)
                dd.run(data)
                dd.plot()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

class Parser():
    def __init__(self):
        self.batch_size = 16
        self.test_batch_size = 1
        self.epochs = 100
        self.lr = 0.001
        self.gamma = 0.7
        self.log_interval = 1
        self.save_model = False
        

def main():
    # Training settings
    args = Parser()
    
    if True:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if device == torch.device("cuda"):
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('example_data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('example_data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = DecentNet().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()
        
        model.update() 

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.ckpt")

    return model

if False:
    model = main()

    for i in model.parameters():
        print(i.shape)
    

# conv filter test

In [ ]:
import torch
import torch.nn.functional as F

# this is one filter

w_groups = 1 # groups of the channels
w_channels = 1024 # input channels
w_filters = 10

assert w_filters % w_groups == 0
assert w_channels % w_groups == 0

# batch size x channels (= w_groups*w_channels) x width x height
inputs = torch.autograd.Variable(torch.randn(27,w_groups*w_channels,100,100))

# w_filters x w_channels x kernel x kernel
weights = torch.autograd.Variable(torch.randn(w_filters,w_channels,3,3))

# batch size x w_filters x width x height
out = F.conv2d(inputs, weights, padding=1, groups=w_groups)

import random



print(inputs.shape, "- batch x groups*channels x width x height")
print(weights.shape, "- filters x channels x kernel x kernel")
print(out.shape, "- batch x filters x width x height")
print()

print("*"*50)

# batch size x channels (= w_groups*w_channels) x width x height
inputs = torch.autograd.Variable(torch.randn(27,w_groups*w_channels,100,100))

output_list = []

# for each filter, we need different true false vales for our channels
active = list(np.random.choice([True, False], size=w_channels, replace=True, p=None))

# w_filters x w_channels x kernel x kernel
weights = torch.autograd.Variable(torch.randn(1,w_channels,3,3))

import time



output_list = []
start = time.time()
for i in range (10):
    for _ in range(w_filters):
        
        pass

        #print()
        #print(active)
        #print()
        #print(inputs.shape, "- batch x groups*channels x width x height")
        #print(weights.shape, "- filters x channels x kernel x kernel")


        # need to remove weight and input channels according to active list for each filter
        #print(inputs.shape)
        #print(inputs[:,active,:,:].shape)

        #print(weights.shape)
        #print(weights[:,active,:,:].shape)




        # batch size x w_filters x width x height
        

        
            



            #output_list.append(this_output)
        #print(this_output.shape, "- batch x 1 filter x width x height")

    #out = torch.cat(output_list, dim=1)

end = time.time()
print(end - start)



weights = torch.autograd.Variable(torch.randn(w_filters,w_channels,3,3))


start = time.time()
for i in range (10):
    i_tmp = inputs[:,active,:,:]
    w_tmp = weights[:,active,:,:]
    this_output = F.conv2d(i_tmp, w_tmp, padding=1, groups=w_groups)
end = time.time()
print(end - start)




weights = torch.autograd.Variable(torch.randn(w_filters,w_channels,3,3))

start = time.time()
for i in range (10):
    this_output = F.conv2d(inputs, weights, padding=1, groups=w_groups)
end = time.time()
print(end - start)



print()
print(out.shape, "- batch x filters x width x height")
print()
print()






# take the mean of all - we can remove all sorts of information from the out tensor
mean = torch.mean(out, 1, keepdim=True)
print(mean.shape, "- mean accross the filters (no sense here ...)")

In [ ]:
import sys
import torch

#b = torch.empty(w_filters, w_channels, dtype=torch.bool)
b = torch.ByteTensor(500, w_channels)
print(sys.getsizeof(b.storage())) # 1310776 (bytes)

#a = torch.empty(w_filters, w_channels, dtype=torch.uint8)
a = torch.ByteTensor(500, w_channels)
print(sys.getsizeof(a.storage())) # 1310776 (bytes)

active = list(np.random.choice([True, False], size=w_channels, replace=True, p=None))
print(sys.getsizeof(active)*500) # 1310776 (bytes)

weights = torch.FloatTensor(torch.randn(500,w_channels,3,3))
print(sys.getsizeof(weights.storage())) # 1310776 (bytes) 36912
print(36912*500)

In [ ]:
weights

In [ ]:
np.random.randint(2, size=10)

list(np.random.choice([True, False], size=10, replace=True, p=None))

In [ ]:
np.random.randint(1, 82, size=200)

In [ ]:
9*9

In [ ]:
np.random.choice(a=4, size=2, replace=False)

In [ ]:
import torch
import torch.nn.functional as F

# this is one filter

w_groups = 27 # groups of the channels
w_channels = 7 # input channels
w_filters = 200
batch_size = 27

# batch size x channels (= w_groups*w_channels) x width x height
inputs = torch.autograd.Variable(torch.randn(batch_size, w_channels*w_groups, 100,100))
#inputs = torch.autograd.Variable(torch.randn(w_channels*w_groups, 100,100))

# w_groups x w_channels x kernel x kernel
weights = torch.autograd.Variable(torch.randn(1, w_channels,3,3))
#weights = torch.autograd.Variable(torch.randn(w_groups*w_channels,3,3))

print(inputs.shape, "- batch x groups*channels x width x height")
print(weights.shape, "- filter 1 x channels x kernel x kernel")

try:
    o_list = []
    for _ in range(w_filters):
        # batch size x groups x width x height
        out = F.conv2d(inputs, weights, groups=w_groups)
        o_list.append(out)
        # take the mean of all - we can remove all sorts of information from the out tensor
        #mean = torch.mean(out, 1, keepdim=True)
    
    print(torch.cat(o_list, dim=1).shape, "- batch x filters x width x height")
    #print(mean.shape, "- mean accross the groups")
except Exception as e:
    print(e)



In [ ]:
filters = torch.randn(8, 4, 3, 3)
inputs = torch.randn(1, 4, 5, 5)
F.conv2d(inputs, filters, padding=1)

# Visualise filters and channels

In [ ]:
model = Net() # .to("cuda")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import utils

def visChannels(tensor, ch=0, allkernels=False, nrow=8, padding=1): 
    n,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(n*c, -1, w, h)
    elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    
    plt.figure(figsize=(nrow,rows) )
    plt.title(f"Channels with index {ch}")
    plt.imshow(grid.detach().cpu().numpy().transpose((1, 2, 0)))


def visFilters(tensor, filt=0, allkernels=False, nrow=8, padding=1): 
    f,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(f*c, -1, w, h)
    elif f != 3: tensor = tensor[filt,:,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure( figsize=(nrow,rows) )
    plt.title(f"Filter {filt}")
    plt.imshow(grid.detach().cpu().numpy().transpose((1, 2, 0)))

def visFilters_subplot(subplot, tensor, filt=0, allkernels=False, nrow=8, padding=1): 
    f,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(f*c, -1, w, h)
    elif f != 3: tensor = tensor[filt,:,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    # plt.figure( figsize=(nrow,rows) )
    subplot.set_title(f"Filter {filt+1} with {c} channels")
    subplot.imshow(grid.detach().cpu().numpy().transpose((1, 2, 0)))
    subplot.axis('off')
    
layer = 1
filter = model.conv2.weight.data.clone()

print(model.conv2.weight.shape)

# need to match the network parameters!!!!
in_channels = 5
out_filters = 3 # 64


fig, subplot = plt.subplots(out_filters, figsize=(10, 10))
fig.suptitle(f'Layer with shape {list(model.conv2.weight.shape)} [out, in, kernel, kernel]')

for filt in range(0, out_filters):
    
    visFilters_subplot(subplot[filt], filter, filt=filt, allkernels=False)

    #plt.axis('off')
    #plt.ioff()
    
plt.tight_layout()
plt.savefig(f"example_results/filter_with_weights.png")
plt.show()
    
if False:    
    for filt in range(0, out_filters):

        visFilters(filter, filt=filt, allkernels=False)

        plt.axis('off')
        plt.ioff()
        plt.tight_layout()
        #plt.savefig(f"examples/example_results/filter_with_weights.png")
        plt.show()

    for ch in range(0, in_channels):

        visChannels(filter, ch=ch, allkernels=False)

        plt.axis('off')
        plt.ioff()
        plt.show()

In [ ]:
import torchvision
res = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

In [ ]:
res.layer1[0].conv1.bias == None

In [ ]:
res.layer1[0].conv1.weight.shape

In [ ]:
res.extra_repr